In [1]:
%cd ..

/home/kirill/PycharmProjects/pythonProject/RecoServiceTemplate


In [2]:
import pandas as pd
from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender
from rectools.dataset import Dataset
from rectools import Columns
from rectools.models.popular import PopularModel
from models.userknn import UserKnn
from rectools.model_selection import TimeRangeSplit
from config.config_models import UserKnn_model_conf

In [3]:
# Train model flag
RETRAIN = False

# Get KION dataset

In [4]:
interactions = pd.read_csv('{0}/interactions.csv'.format(UserKnn_model_conf.dataset_path))
users = pd.read_csv('{0}/users.csv'.format(UserKnn_model_conf.dataset_path))
items = pd.read_csv('{0}/items.csv'.format(UserKnn_model_conf.dataset_path))

In [5]:
# rename columns, convert timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight},
                    inplace=True)

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

In [6]:
max_date = interactions['datetime'].max()
min_date = interactions['datetime'].min()

# From `itemkNN` to `userkNN `

In [7]:
n_folds = UserKnn_model_conf.n_folds
unit = UserKnn_model_conf.unit
n_units = UserKnn_model_conf.n_units
periods = UserKnn_model_conf.periods
freq = f"{n_units}{unit}"
last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)

date_range = pd.date_range(start=start_date, periods=periods, freq=freq,
                           tz=last_date.tz)
# generator of folds
cv = TimeRangeSplit(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [8]:
# we have just 1 test fold - no need to iterate over fold
(train_ids, test_ids, fold_info) = cv.split(interactions, collect_fold_stats=True).__next__()

# Fit userKNN model

In [9]:
train = interactions.loc[train_ids]
test = interactions.loc[test_ids]

In [10]:
import dill

model = UserKnn_model_conf.model

# Train model flag
if RETRAIN:
    userknn_model = UserKnn(model=model, N_users=50)
    userknn_model.fit(train)
else:
    with open(UserKnn_model_conf.weight_path, 'rb') as f:
        userknn_model = dill.load(f)

# Get recomendations

In [11]:
userknn_reco_df = userknn_model.predict(test, UserKnn_model_conf.N_recs)

In [12]:
userknn_reco_df[userknn_reco_df.user_id == 1097544]

,user_id,item_id,score,rank
0,1097544,6309,5.979983,1
3,1097544,12768,4.52878,2
9,1097544,12455,3.681267,3
2,1097544,14317,2.907646,4
8,1097544,7626,2.633196,5
7,1097544,12173,2.621315,6
12,1097544,5658,2.470598,7
10,1097544,7829,2.401344,8
13,1097544,4740,2.040055,9
14,1097544,9996,2.03397,10


# Save recomendations df

In [13]:
userknn_reco_df.to_csv(UserKnn_model_conf.save_reco_df_path, encoding='utf-8', index=False)

# df = pd.read_csv('userknn_reco_df', encoding='utf-8')


In [13]:
# # save model
import dill

with open(UserKnn_model_conf.weight_path, 'wb') as f:
    dill.dump(userknn_model, f)